# Assignment 1: Sentiment Polarity for Film Reviews

*version 2024.2, details/bugs in this assignment might be patched, you will be notified if this happens*

For this assignment, you will implement a classification pipeline for a binary sentiment classification task. The provided dataset includes film reviews, labelled as either positive or negative. Your starting point for the assignment is an implementation of the full pipeline that uses scikit learn and example data.

In most real-world situations, you will have good (and relatively bug free) tools at your disposal. However, for being able to be creative with an ML problem, it is crucial to understand the inner workings of a full pipeline. You will therefore re-implement the model, feature extraction, the learning algorithm, the prediction algorithm, and a simple grid search for hyperparameters. You will find re-implementation tasks marked "implementation task" (in bold) below. In your submission, all code must be written from scratch by you. While you can use the base modules of python (os, string etc) and NumPy, you cannot use sklearn or any equivalent ML library (when in doubt, ask about imports). It is recommended to use the reference pipeline while implementing your own functions. This way, you can try your own code throughout, without having to get all the moving parts in place before testing.

## Submission

Please submit your code as a notebook through studium. You should include the following:

1. A working implementation of your pipeline, reproducing your principal results when run. Please rerun your notebook as the last thing you do before submitting.
2. Comment the code properly, especially for longer or opaque functionality. Please try to write self documenting code (i.e., choosing descriptive variables names, refactoring to isolate functionality, minimal code duplication etc).
3. A *brief* description (100-200 words in total) of the implementation work that was necessary to complete the different parts, showing how you arrived at your solution and design choices made. You can spread these in the notebook or put them in one place.
4. Comments on what you thought was hard in the assignment, what you think was educational, what took most time and which parts might be unnecessarily tricky.
5. As submissions are anonymous, **all personal information must be removed**. This is a hard requirement.
6. Apart from section titles, please remove all unnecessary text and code from the notebook you hand in. Keep only that which strengthens the case that you fulfil the listed requirements. All notebooks containing unnecessary chunks of text from this instruction will receive a failing grade (U).

## Requirements for grade G

To achieve a pass (G) in this assignment, you must solve the following tasks without serious errors.

1. Reimplement the four parts of the assignment. The instructions below will guide you on specifics for each part.
2. When evaluating your model, split the data into a training and a test set. This split can be selected non-randomly before running any training. The split should be 80/20 (i.e., 1600 training documents and 400 test documents). *It is easier for you if the classes are balanced.*
3. Your code should work as plug-in replacements for the vectorizer and model. Hence, you should implement the model and the vectorizer as classes following the sklearn API as:

```
class feature_transform:
  def __init__(self, ...):
    ...

  def fit_transform(self, X):
    ...


class model:
  def __init__(self, learning_rate, ...):
    ...

  def fit(self, X, y):
    ...

  def predict(self, X):
    ...

  def score(self, X, y):
    ...
```
4. Include a short (150 words) analysis in your submission. Discuss the final decision boundary (i.e., why did some word contribute to a negative/positive label), the selected hyperparameters, and design choices. Note that hyperparameters do not need to be systematically selected, use something that works.

### Additional requirements for grade VG

To achieve a pass with distinction (VG) in this assignment, you must adequately solve the tasks above for a passing grade (G). In addition, you must:

1. Implement the optimization as *stochastic* gradient descent (SGD)
2. Implement a [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) feature model, and compare classification performance to bag-of-words (this should also be briefly discussed in your analysis). Choose your preferred formulation of tf-idf from the literature, *briefly* motivating your choice.
3. Implement an extension to the SDG optimization of your choice, e.g. from [this list on wikipedia](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Extensions_and_variants).
4. Implement [k-fold cross validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation) for evaluating and comparing your model variants.
5. Prepare a presentation (~5min) with analysis of your design choices, pipelines, and results. How much did you gain in performance by using the more complex pipelines? The analysis and claims must be essentially correct. Submit and handful of slides in pdf format with your notebook.

## General advice

The task is to predict, for an unseen review, whether it is positive or negative. This is a binary classification task. Work from the given code and change one piece at a time. You should test your code every couple of lines to make sure your assumptions on functionality and variable content are correct. A good rule of thumb is that a coder will introduce a bug every five lines (even as a professional).

To make sure your code does what it is supposed to do, use ```assert``` statements to check your assumptions. Keep the given asserts if you need them. Professional coders sometimes start with writing tests for some functionality instead of starting with the functionality itself. This is called *test-driven development*.

When developing, it is often best to start with *simple and clear code*, then going on to think about efficiency. Writing efficient code will not help you if you've missunderstood parts of the task description. For example, when finding the gradient, start by creating the vector and running a `for` loop over the data. Only then do you go on to experiemnt with vector operations and sparse matrices (assuming you want to).

## Plagiarism

In code assignments, plagiarism is a tricky concept. A clean cut way would be to demand that you write all the code yourself, from memory, with only the assigned literature as help. This is not how code is developed professionally, where it is common to copy and share. However, since this is a learning exercise, you must implement everything on your own, but please look at the course repo, Stack Overflow etc. Moreover, discuss with course mates and TAs to find inspiration and solutions. Code that is *obviously* copied (with minor modifications) will be considered as plagiarized. As a rule of thumb, if you can explain a solution to some problem in a brief discussion, you're safe. If you share your code, even with the best of intentions (or due to social pressure), the submitted (or committed) code with the earliest time stamp will be considered as the original. As a part of the examination, you might be asked to explain any particular part of the functionality in your implementation.

## Part 1: Parsing the dataset

For this assignment, we use the [Review polarity v2.0](http://www.cs.cornell.edu/people/pabo/movie-review-data/) data set created by Bo Pang and Lillian Lee at Cornell University. It consists of 2000 movie reviews, 1000 of which are positive and 1000 are negative. *Always check the readme for any dataset before using it.*

The following downloads the dataset (if it's not already present).

In [ ]:
import os.path
import tarfile
from urllib.request import urlretrieve

url = """http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz"""
if not os.path.exists(url.split("/")[-1]):
  urlretrieve(url, url.split("/")[-1])
  print("Downloaded", url)
  with tarfile.open('review_polarity.tar.gz', 'r:gz') as tar:
    tar.extractall()
  print("Extracted archive")

In order to get the given code to work, a part of the 20 newsgroup dataset is loaded below. This should be replaced by your parsing code in your final submission.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
X_raw, y = fetch_20newsgroups(subset='train', categories=['rec.sport.baseball', 'rec.sport.hockey'], remove=('headers', 'footers', 'quotes'), return_X_y=True)
y[y==0]=-1

In [ ]:
import numpy as np

# assert len(X_text) == 2000
assert np.all([isinstance(x, str) for x in X_raw])
assert len(X_raw) == y.shape[0]
assert len(np.unique(y))==2
assert y.min() == -1
assert y.max() == 1

**Implementation task:** Implement a parser for the dataset. The output should be a list/array of strings (`X_raw`) and a list/array of labels (`y`) encoded as {-1,1}.



## Part 2: Feature extraction

As basic features, we use a binary bag-of-words (BOW) representation of the words in each review. Each review in the data set is described by a vector with one element corresponding to each word in the vocabulary. An element is set to 1 if the review contains its associated word, otherwise it is set to 0.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(X_raw)             # Creates the vocabulary of the vectorizer

X = vectorizer.transform(X_raw)
X = X.todense()                   # sklearn will output a sparse matrix
X[X>1] = 1                        # Turns the count vectors into binary vectors
X = np.asarray(X)                 # Turns the matrix into an array. SGD model doesn't support matrix in the newer sklearn module version

ordered_vocabulary = vectorizer.get_feature_names_out()
vocabulary = set(ordered_vocabulary)

If needed, we can do lookup tables going from tokens to feature numbers. Note that most of the elements in any feature vector will be zero.

In [ ]:
lut = dict()
for i, word in enumerate(ordered_vocabulary):
  lut[word] = i

for word in ['dolphin', 'the', 'coffee']:
  if word in vocabulary:
    print("'%s' is represented as feature dimension %i" %(word, lut[word]))
  else:
    print("'%s' is not in the vocabulary" % word)

**Implementation task:** You should re-implement the feature extraction above. The list/array called `ordered_vocabulary` should contain the words for each feature dimension, and X should contain the BOW binary vectors. Remember to use the same method names as the original sklearn class.

*Hints: Implementing X as a NumPy array or a SciPy sparse matrix (not as a list) will make your life easier in the coming parts. Also, the `in` operator is way faster for sets than for lists.*

We can now look at the data and the words corresponding to feature dimensions.

In [ ]:
print(ordered_vocabulary[2000:2010])
print(X[:10, 2000:2010])
for w in ['dolphin', 'the', 'coffee']:
  print("'%s' in words: %s" % (w, w in vocabulary))

At this point in the code, X and y are ready for the supervised learning task.

## Part 3: Learning framework

The model we will use is a simple hyperplane. This plane will represent the decision boundary through the data space, separating positive from negative ratings. Note that the following three parts are independent, and can be implemented in separate methods.

**Implementation task:** You should implement your versions of the following parts (you can also find this in the slides):

1. **Hyperplane model**. The model should be a hyperplane as $f(x, \omega) = sgn(\omega^\top x)$, where $sgn(\cdot)$ is the [sign function](https://en.wikipedia.org/wiki/Sign_function). Note that $x_0$ in this notation is the pseudo input 1. When evaluated, this gives us the predicted results as $\hat y_i = f(X_i, \omega^{(t)})$, where $\omega^{(t)}$ is the parameter vector at optimization iteration t and $\omega^{(0)}$ is the initial guess for the parameter vector.

2. **Objective function.** The loss function for our model is the [hinge loss](https://en.wikipedia.org/wiki/Hinge_loss), which will be used together with $l_2$ regularization.

  $\mathfrak{L}(X, y, \omega) = \frac{\lambda}{2} ||\omega||^2 + \sum_{i=1}^{|X|} \max(0, 1-y_i \cdot \omega^\top X_i)$.

  Regularization is done by adding a norm on the parameter vector and including that in the objective function. A shorter parameter vector gives a larger margin for this model. The $l_2$ norm is defined as $\sqrt{\sum_{i=1}^n \omega_i^2}$. The regularization always has some positive attenuation parameter $\lambda \in \mathbb{R}$ keeping it from dominating the objective function. It symbolizes a trade-off between a more accurate classification and wider margins, while also giving the objective function a unique solution.

3. **Gradient descent**. The update for gradient descent looks like $\omega^{(t)} = \omega^{(t-1)} - \gamma \nabla \mathfrak{L}(\omega^{(t-1)})$, where the update gradient is defined as $\nabla \mathfrak{L}(X, y, \omega) = \left ( \frac{\partial \mathfrak{L}(X, y, \omega)}{\partial \omega_0}, \frac{\partial \mathfrak{L}(X, y, \omega)}{\partial \omega_1}, \ldots, \frac{\partial \mathfrak{L}(X, y, \omega)}{\partial \omega_n} \right )^\top$. The expression for this gradient $\nabla \mathfrak{L}$ is given analytically as:

  $\nabla \mathfrak{L}(X, y, \omega) = \lambda \omega + \sum_{i=1}^{|X|}
\begin{cases}
0 & \text{if } y_i \omega^\top X_i \geq 1\\
-y_i X_i & \text{else}
\end{cases}$

In the expression for the gradient, $X_i$ is a vector (with pseudo input) and $y_i$ is a scalar. These two refer to the i:th data point and its label. The learning rate $\gamma \in \mathbb{R}$ acts as a scaling/dampening factor on the gradient update. This should run until some stopping criteria is met (e.g., $\omega^{(t+1)}\approx \omega^{(t)}$). In `SGDClassifier` from sklearn, this is interpreted as finding $loss_{current} > loss_{best} - .001$ for five consecutive iterations.

*Note that while your code will be runnable, it will likely be much slower than sklearn's code.*

In [ ]:
from sklearn.linear_model import SGDClassifier

# Set hyperparameters (these variables are only here for clarity)
reguliser_dampening = 0.001   # lambda
learning_rate = .1            # gamma

# Create the untrained classifier
model = SGDClassifier(loss='hinge', penalty='l2',
                      alpha=reguliser_dampening, verbose=1,
                      learning_rate='constant', eta0=learning_rate)

# Train the classifier
model.fit(X, y)

# Get the parameter vector
omega = np.concatenate([model.intercept_, model.coef_.ravel()])

In the above training with verbose=1, note how the loss etc are changing. For your implementation, it can be very good to print out lots of information so that you can see if you get what you expect (e.g. a lowering of the loss).

We can examine the weights by plotting them. Think about how to interpret these weights.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 3))
plt.plot(omega[1:])
plt.xlabel("Value")
plt.xlabel("Weights")
plt.show()

From the same information, we can plot the words with the strongest influence. Can you see a pattern? Is the model over learning on some words? Why might word frequency be important when analysing their impact?



In [ ]:
assert (len(omega)-1) == len(vocabulary)

# Sort by absolute value
idx = np.argsort(np.abs(omega[1:]))

print("                Word   Weight  Occurences")
for i in idx[-20:]:   # Pick those with highest 'voting' values
  print("%20s   %.3f\t%i " % (ordered_vocabulary[i], omega[i+1], np.sum([ordered_vocabulary[i] in d for d in X_raw])))

## Part 4: Exploring hyperparameters

For optimization of the hyperparameters, you can search for values on a grid. Trying all combinations is called a grid search and can be implemented with nested `for` loops. A faster alternative is to sample from the grid. Sampling is not as thorough, but most often sufficient (and much faster). Following the current consensus on ML methodology, we must split off a test set before exploring any configuration and use this data only at the very end of the experiment. You can use sklearn for the hyperparameter search.

**Implementation task:** Implement code for printing a sorted table of your sampled hyperparameters. Note, you do not have to reimplement the grid search.

1. **Learning rate**. Try different settings of the learning rate. It is useful to pick values from an exponentially spaced grid (e.g., 0.0001/0.0003/0.001/0.003/0.01/0.03/0.1/0.3/1.0/3.0) where each of the values is about 3 times as large as the previous one. Note what happens when the learning rate gets too small or too large. The best learning rate is as large as possible, but still reliable and stable.

3. **Regulariser dampening**. Try to find how much the regulariser needs to be dampened to get a good score. Use an exponentially spaced grid here as well.


The grid points can be generated as `np.exp(np.linspace(np.log(0.0001), np.log(3), 10))`, and pick a random subset by running `np.random.choice(your_array, size=10)` (the code below generates 100 grid points but only tries 10 of them). For these real valued hyperparameters, you can add extra points if you find a promising interval. Note that trying every combination of hyperparameters is likely not feasible, but don't hesitate to try.

Once you’ve settled on a final set of hyperparameters that work well/decently (and only then!), use the test set and rerun to obtain an accuracy score on the test set.

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting of the test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.model_selection import ParameterSampler

# Define the parameters
parameter_distribution = {'learning_rate': np.exp(np.linspace(np.log(0.0001), np.log(3), 10)),
                          'reguliser_dampening': np.exp(np.linspace(np.log(0.0001), np.log(3), 10))}

# Placeholder to make future comparissons easier
best_hyperparameters = None
print("Learning rate:\tReg.dampening:\tTraining set accuracy:")

for hyperparameters in ParameterSampler(parameter_distribution, n_iter=10):
  # Set up the classifier
  reguliser_dampening = hyperparameters['reguliser_dampening']
  learning_rate = hyperparameters['learning_rate']
  model = SGDClassifier(loss='hinge', penalty='l2',
                        alpha=reguliser_dampening, verbose=0,
                        learning_rate='constant', eta0=learning_rate)

  # Train the classifier
  model.fit(X_train, y_train)

  # Calculate the training accuracy
  training_accuracy = np.sum(model.predict(X_train)==y_train)/len(y_train)

  # Store the hyperparameters if they are better than what we have found before
  if best_hyperparameters is None or best_hyperparameters[1] < training_accuracy:
    best_hyperparameters = (hyperparameters, training_accuracy)
  print("%.5f\t\t%.5f\t\t%.1f%%" % (hyperparameters['learning_rate'], hyperparameters['reguliser_dampening'], 100*training_accuracy))

best_learning_rate = best_hyperparameters[0]['learning_rate']
best_reguliser_dampening = best_hyperparameters[0]['reguliser_dampening']
print("Best parameters: %.5f, %.5f" % (best_learning_rate, best_reguliser_dampening))

We can now run the final model.

In [ ]:
# Set up the classifier
model = SGDClassifier(loss='hinge', penalty='l2',
                      alpha=best_reguliser_dampening, verbose=0,
                      learning_rate='constant', eta0=best_learning_rate)

# Train on all the non-test data
model.fit(X_train, y_train)

# Run prediction on the test set
test_accuracy = np.sum(model.predict(X_test)==y_test)/len(y_test)

print("Test set accuracy %.1f%%" % (100*test_accuracy))

Is this a good result? What is a good result?

